In [74]:
import requests
from bs4 import BeautifulSoup
import nest_asyncio
import asyncio
import aiohttp
import pandas as pd
import re

In [75]:
df = pd.read_csv('basic_info.csv', dtype={'tickers': str})
indicators = [
    'diluted_earnings_per_share', 'weighted_earnings_per_share', 'net_asset_per_share', 
    'return_on_equity', 'operating_profit_margin', 'net_profit_margin', 
    'return_on_assets', 'current_ratio', 'quick_ratio', 'cash_ratio', 'debt_to_asset_ratio'
]

# datas
dates = ['06/30/24', '03/31/24', '12/31/23', '09/30/23']

for indicator in indicators:
    for date in dates:
        df[f'{indicator} {date}'] = None

In [76]:
def extract_indicator(soup, indicator_name):
    indicator_row = soup.find(text=re.compile(indicator_name))

    if indicator_row:
        # Extract the adjacent td elements of this row, limited to 4
        return [td.text.strip() for td in indicator_row.find_all_next('td', limit=4)]
    else:
        return None

# Loop through each stock code
for index, row in df.iterrows():
    stock_code = row['tickers']
    url = f"https://vip.stock.finance.sina.com.cn/corp/go.php/vFD_FinancialGuideLine/stockid/{stock_code}/displaytype/4.phtml"
    
    # request
    source = requests.get(url)
    soup = BeautifulSoup(source.text, 'lxml')
    
    # Extract data for all indicators
    indicators = {
        "摊薄每股收益": ['diluted_earnings_per_share 06/30/24', 'diluted_earnings_per_share 03/31/24', 'diluted_earnings_per_share 12/31/23', 'diluted_earnings_per_share 09/30/23'],
        "加权每股收益": ['weighted_earnings_per_share 06/30/24', 'weighted_earnings_per_share 03/31/24', 'weighted_earnings_per_share 12/31/23', 'weighted_earnings_per_share 09/30/23'],
        "每股净资产_调整后": ['net_asset_per_share 06/30/24', 'net_asset_per_share 03/31/24', 'net_asset_per_share 12/31/23', 'net_asset_per_share 09/30/23'],
        "净资产收益率": ['return_on_equity 06/30/24', 'return_on_equity 03/31/24', 'return_on_equity 12/31/23', 'return_on_equity 09/30/23'],
        "营业利润率": ['operating_profit_margin 06/30/24', 'operating_profit_margin 03/31/24', 'operating_profit_margin 12/31/23', 'operating_profit_margin 09/30/23'],
        "销售净利率": ['net_profit_margin 06/30/24', 'net_profit_margin 03/31/24', 'net_profit_margin 12/31/23', 'net_profit_margin 09/30/23'],
        "总资产利润率": ['return_on_assets 06/30/24', 'return_on_assets 03/31/24', 'return_on_assets 12/31/23', 'return_on_assets 09/30/23'],
        "流动比率": ['current_ratio 06/30/24', 'current_ratio 03/31/24', 'current_ratio 12/31/23', 'current_ratio 09/30/23'],
        "速动比率": ['quick_ratio 06/30/24', 'quick_ratio 03/31/24', 'quick_ratio 12/31/23', 'quick_ratio 09/30/23'],
        "现金比率": ['cash_ratio 06/30/24', 'cash_ratio 03/31/24', 'cash_ratio 12/31/23', 'cash_ratio 09/30/23'],
        "资产负债率": ['debt_to_asset_ratio 06/30/24', 'debt_to_asset_ratio 03/31/24', 'debt_to_asset_ratio 12/31/23', 'debt_to_asset_ratio 09/30/23']
    }

    #Iterate through each indicator, extract data, and update the DataFrame
    for indicator, cols in indicators.items():
        values = extract_indicator(soup, indicator)
        
        if values and len(values) == 4:
            df.at[index, cols[0]] = values[0]
            df.at[index, cols[1]] = values[1]
            df.at[index, cols[2]] = values[2]
            df.at[index, cols[3]] = values[3]
        else:
            print(f"Warning: Missing values for {indicator} for stock {stock_code}")


/var/folders/1g/kkn6qdxn5jj1lnwwtw1hhwg00000gn/T/ipykernel_39926/329794052.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  indicator_row = soup.find(text=re.compile(indicator_name))


In [77]:
# for index, row in df.iterrows():
#     # Generate URLs dynamically (replace with stock codes)
#     stock_code = row['tickers']
#     # print(stock_code)
#     url = f"https://vip.stock.finance.sina.com.cn/corp/go.php/vFD_FinancialGuideLine/stockid/{stock_code}/displaytype/4.phtml"
    
#     # request
#     source = requests.get(url)
#     # Check if the request was successful
#     # if source.status_code == 200:
#     #     print("Request was successful, proceed with data processing")
#     #     # Process the data, for example, using BeautifulSoup to parse
#     #     # soup = BeautifulSoup(response.text, 'html.parser')
#     # else:
#     #     print(f"Request failed with status code: {source.status_code}")
#     soup = BeautifulSoup(source.text, 'lxml')

# # 查找摊薄每股收益的行
#     diluted_eps_row = soup.find(text=re.compile("摊薄每股收益"))

#     # 检查是否找到该行，避免抛出 AttributeError
#     if diluted_eps_row:
#         # 提取所有相邻的 td 元素（假设这些值在连续的 td 中）
#         eps_values = [td.text.strip() for td in diluted_eps_row.find_all_next('td', limit=4)]
        
#         # 将数据填充到 DataFrame
#         if len(eps_values) == 4:
#             df.at[index, 'diluted_earnings_per_share 06/30/24'] = eps_values[0]
#             df.at[index, 'diluted_earnings_per_share 03/31/24'] = eps_values[1]
#             df.at[index, 'diluted_earnings_per_share 12/31/23'] = eps_values[2]
#             df.at[index, 'diluted_earnings_per_share 09/30/23'] = eps_values[3]
#         else:
#             print(f"Warning: Missing values for stock {stock_code}")
#     else:
#         print(f"Warning: Could not find '摊薄每股收益' for stock {stock_code}")

#     # diluted_earnings_per_share1 = soup.find(text=re.compile("摊薄每股收益")).find_next('td').text.strip()
#     # diluted_earnings_per_share2 = soup.find(text=re.compile("摊薄每股收益")).find_next('td').find_next('td').text.strip()
#     # diluted_earnings_per_share3 = soup.find(text=re.compile("摊薄每股收益")).find_next('td').find_next('td').find_next('td').text.strip()
#     # diluted_earnings_per_share4 = soup.find(text=re.compile("摊薄每股收益")).find_next('td').find_next('td').find_next('td').find_next('td').text.strip()

#     # df.at[index, 'diluted_earnings_per_share 06/30/24'] = diluted_earnings_per_share1
#     # df.at[index, 'diluted_earnings_per_share 03/31/24'] = diluted_earnings_per_share2
#     # df.at[index, 'diluted_earnings_per_share 12/31/24'] = diluted_earnings_per_share3
#     # df.at[index, 'diluted_earnings_per_share 09/30/24'] = diluted_earnings_per_share4



In [82]:
missing_values_count = df.isnull().sum()
print(missing_values_count)

tickers                                 0
company                                 0
listed_market                           0
listing_date                            0
issue_price                             1
company_website                         0
business_address                        0
main_business                           0
diluted_earnings_per_share 06/30/24     0
diluted_earnings_per_share 03/31/24     0
diluted_earnings_per_share 12/31/23     0
diluted_earnings_per_share 09/30/23     0
weighted_earnings_per_share 06/30/24    0
weighted_earnings_per_share 03/31/24    0
weighted_earnings_per_share 12/31/23    0
weighted_earnings_per_share 09/30/23    0
net_asset_per_share 06/30/24            0
net_asset_per_share 03/31/24            0
net_asset_per_share 12/31/23            0
net_asset_per_share 09/30/23            0
return_on_equity 06/30/24               0
return_on_equity 03/31/24               0
return_on_equity 12/31/23               0
return_on_equity 09/30/23         

In [81]:
df.to_csv('financial_report_data.csv', index=False)